# Imports

In [4]:
# common / KNN imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import scipy.spatial.distance
from collections import Counter
plt.set_cmap('gray')

# Aufgaben

## A KNN

#### How many distances you need to calculate if you have 60,000 samples in the trainingset for 50 samples?
Man muss 3.000.000 Abstände berechnen.
#### How many distances do you need to calculate if you have n samples im the trainingset?
Man muss immer n * 50 Abstände berechnen, d.h. bei 3 Samples müsste man 150 Abstände berechnen, bei 90.000 dann 4.500.000

## B KNN

In [9]:
def knn(k, x_train, y_train, x_test):
    dists = scipy.spatial.distance.cdist(x_train, x_test, metric='euclid')
    idx_nearest = np.argpartition(dists, k, axis=0)[:k]
    idx_nearest = np.swapaxes(idx_nearest, 0, 1)
    #print(idx_nearest)

    closest_points_labels = y_train[idx_nearest]
    y_test_predictions = []
    
    for index, predicted_labels in enumerate(closest_points_labels):
        y_test_predictions.append(np.bincount(predicted_labels).argmax())
    return y_test_predictions

In [10]:
def knn_print_error_rate(y_test,  y_test_predictions):
    
    correct = 0
    
    correct_labels = (y_test)
    
    numbers_count = Counter(correct_labels)
    
    numbers_correct = dict.fromkeys(numbers_count.keys(), 0)
    
    #numbers_error_rate = dict.fromkeys(numbers_count.keys(), None)
    
    
    n = correct_labels.shape[0]
    
    for index, prediction in enumerate( y_test_predictions):
        if prediction == correct_labels[index]:
            correct += 1
            numbers_correct[prediction] += 1
        
    error_rate = 1- (correct / n)
    
    
    for key in sorted(numbers_count):
        count = numbers_count[key]
        correct = numbers_correct[key]
        error = 1 - numbers_correct[key] / numbers_count[key]
        print("Label:", key)
        print("  ", correct, "out of", count, "recognized correctly | Errorrate:", error*100, "%")
        
    print("\n"+"Resulting errorrate:", error_rate*100, "%")

In [11]:
#Load dataset
mnist_data = tf.keras.datasets.mnist.load_data()

# Get Training set
train = mnist_data[0]
X_train, Y_train = train[0], train[1]

# Get Test set
test = mnist_data[1]
X_test, Y_test = test[0], test[1]

n_train = X_train.shape[0]
n_test = X_test.shape[0]

m = X_train.shape[1]**2

X_train = X_train.reshape([n_train, m])
X_test = X_test.reshape([n_test, m])

# Create train subset
idx = np.random.randint(n_train, size=10000)
x_train_sample = X_train[idx]
y_train_sample = Y_train[idx]

# Create test subset
idx = np.random.randint(n_test, size=30)
x_test_sample = X_test[idx]
y_test_sample = Y_test[idx]

In [12]:
predictions2 = knn(2, x_train_sample, y_train_sample, x_test_sample)
predictions4 = knn(4, x_train_sample, y_train_sample, x_test_sample)
predictions8 = knn(8, x_train_sample, y_train_sample, x_test_sample)

print(list(y_test_sample), "\n")

print("-------------------------------------")
print("  2NN  ")
print("-------------------------------------")
print("Predictions:")
print(predictions2, "\n")
knn_print_error_rate(y_test_sample, predictions2)

print("-------------------------------------")
print("  4NN  ")
print("-------------------------------------")
print("Predictions:")
print(predictions4, "\n")
knn_print_error_rate(y_test_sample,predictions4)

print("-------------------------------------")
print("  8NN  ")
print("-------------------------------------")
print("Predictions:")
print(predictions8, "\n")
knn_print_error_rate(y_test_sample,predictions8)


[1, 9, 3, 5, 6, 6, 7, 7, 1, 1, 9, 5, 3, 8, 2, 5, 7, 8, 7, 0, 6, 6, 9, 1, 8, 1, 0, 4, 1, 7] 

-------------------------------------
  2NN  
-------------------------------------
Predictions:
[1, 9, 3, 5, 6, 6, 7, 7, 1, 1, 9, 3, 3, 8, 2, 0, 7, 1, 7, 0, 6, 6, 9, 1, 8, 1, 0, 4, 1, 7] 

Label: 0
   2 out of 2 recognized correctly | Errorrate: 0.0 %
Label: 1
   6 out of 6 recognized correctly | Errorrate: 0.0 %
Label: 2
   1 out of 1 recognized correctly | Errorrate: 0.0 %
Label: 3
   2 out of 2 recognized correctly | Errorrate: 0.0 %
Label: 4
   1 out of 1 recognized correctly | Errorrate: 0.0 %
Label: 5
   1 out of 3 recognized correctly | Errorrate: 66.66666666666667 %
Label: 6
   4 out of 4 recognized correctly | Errorrate: 0.0 %
Label: 7
   5 out of 5 recognized correctly | Errorrate: 0.0 %
Label: 8
   2 out of 3 recognized correctly | Errorrate: 33.333333333333336 %
Label: 9
   3 out of 3 recognized correctly | Errorrate: 0.0 %

Resulting errorrate: 9.999999999999998 %
----------------

##### What is the error rate of KNN on the test set?
Die Errorrate beim ersten Testen (mit k=2, train = 10.000, test = 30) war 6%.  
Beim testen mit neuen subsets gab es auch Errorraten zwischen 3 - 16%
##### What is the error rate for each label (number)?
Die Errorraten beim ersten Testen (mit k=2) waren:
* 0 - 0.0%
* 1 - 0.0%
* 2 - 0.0%
* 3 - 0.0%
* 4 - 33.0% (2/3)
* 5 - 20.0% (4/5)
* 6 - 0.0%
* 7 - 0.0%
* 8 - 0.0%
* 9 - 0.0%  
  
Beim testen mit anderen Subsets fällt auf, dass es bei den Zahlen 0-2 tendenziell niedrigere Fehlerquoten gibt als beim Rest.
##### How does the choice of k influence the result?
Beim ersten Testen hat sich die Errorrate mit zunehmendem k verbessert:

* k = 2: 6%
* k = 4: 3%
* k = 8: 3%  
  
Beim Testen mit anderen Subsets ist aufgefallen, dass k sich nicht immer auf die Errorquote auswirkt, aber k=4 tendenziell niedrigere Errorraten für unsere Test- und Traingrößen hatte.

## C K-means MNIST

In [13]:
# load subset
n_sample = 1000
n = X_train.shape[0]

idx = np.random.randint(n, size=n_sample)
x_mnist = X_train[idx]
y_mnist = Y_train[idx]

k_mnist = 10
it_mnist = 30
_tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
embedding = _tsne.fit_transform(x_sample)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.023s...
[t-SNE] Computed neighbors for 1000 samples in 1.404s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 609.178494
[t-SNE] KL divergence after 250 iterations with early exaggeration: 67.772636
[t-SNE] KL divergence after 300 iterations: 1.376749


In [32]:
def kmeans_train(k, it, x_train):
    
    # init centroids
    idx_centroid = np.random.randint(x_train.shape[0], size=k)
    centroids = x_train[idx_centroid]
    
    # calc distances
    dists = scipy.spatial.distance.cdist(x_train, centroids, metric='euclid')
    
    # find nearest centroids
    labels = np.argmin(dists, axis=1)

    # train model for q iterations
    for j in range(it):
        
        # calc distances
        dists = scipy.spatial.distance.cdist(x_train, centroids, metric='euclid')
        
        # label according to nearest centroids
        labels = np.argmin(dists, axis=1)
        
        # get new centroids
        centroid_list = [x_train[labels == i].mean(0) for i in range(k)]
        centroids = np.stack(centroid_list)
        

In [33]:
kmeans_train(k_mnist, it_mnist, x_mnist)

TypeError: kmeans_train() missing 2 required positional arguments: 'q' and 'x_train'

In [31]:
# print results

for i in range(k):
    
    size = y_sample[labels == i].shape[0]
    bincount = np.bincount(y_sample[labels == i])
    mode = bincount.argmax()
    correct = bincount[mode]
    percentage = correct / size
    
    print('group', i)
    print("-------------------------------------")
    print("label", mode, "(size:", size, ")")
    print("correct", correct)
    print("size", size)
    print("Percentage of majority class:", percentage*100, "%", "\n")

group 0
-------------------------------------
label 1 (size: 215 )
correct 92
size 215
Percentage of majority class: 42.7906976744186 % 

group 1
-------------------------------------
label 7 (size: 30 )
correct 15
size 30
Percentage of majority class: 50.0 % 

group 2
-------------------------------------
label 2 (size: 90 )
correct 52
size 90
Percentage of majority class: 57.77777777777777 % 

group 3
-------------------------------------
label 7 (size: 101 )
correct 56
size 101
Percentage of majority class: 55.44554455445545 % 

group 4
-------------------------------------
label 4 (size: 75 )
correct 32
size 75
Percentage of majority class: 42.66666666666667 % 

group 5
-------------------------------------
label 5 (size: 82 )
correct 30
size 82
Percentage of majority class: 36.58536585365854 % 

group 6
-------------------------------------
label 3 (size: 96 )
correct 34
size 96
Percentage of majority class: 35.41666666666667 % 

group 7
-------------------------------------
label

#### Do this for 10, 100, 1000 iterations
##### What is the majority class of each cluster?
##### What is the percentage of the majority class in each cluster?
##### Does each number have a cluster?
##### If not, which hasn’t?


## D MNIST understanding

##### Some numbers are more difficult to predict with KNN and to cluster with K-means. Show why.  Start with examples, and get more general

## E KNN and K-means CIFAR

In [ ]:
# Load dataset

cifar = tf.keras.datasets.cifar10.load_data()[0]
x_cifar, y_cifar = cifar[0], cifar[1]

# Convert to grayscale
x_cifar = np.dot(x_cifar[...,:3], [0.299, 0.587, 0.114])
print(x_cifar.shape)

n_x_cifar = x_cifar.shape[0]
m_x_cifar = x_cifar.shape[1]**2

x_cifar = x_cifar.reshape([n_x_cifar, m_x_cifar])

In [ ]:
# Create subset
idx = np.random.randint(n_x_cifar, size=1030)
x_cifar_sample = x_cifar[idx]
y_cifar_sample = y_cifar[idx]

# Create test & train set
x_cifar_test = x_cifar_sample[:30]
y_cifar_test = y_cifar_sample[:30]

x_cifar_train = x_cifar_sample[30:]
y_cifar_train = y_cifar_sample[30:]

print(x_cifar_test.shape)
print(y_cifar_test.shape)
print(x_cifar_train.shape)
print(y_cifar_train.shape)

transform cifar-10 to grayscale.
* Does knn work similarly good?
* Does k-means work similarly good?
* Demonstrate this similar to B, C and D

## F Linear regression BOSTON

Train a linear regression model on the BOSTON dataset to predict median
values of houses. Test it on the test set. Show the code how you did it! What
dependent variables (columns) have the biggest influence?

## G Logistik regression MNIST

Evalute logistic regression as B and D on MNIST

## H Feedback

How much time did you work on this assignment and how often did you
meet?
* Did you use Google Colab or Anaconda?
* What was your favourite excercise? Why?
* What excercise did you like least? Why?